In [1]:
import sys 
# setting path
sys.path.append('../')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import requests
import logging
import knockpy
from knockpy.knockoff_filter import KnockoffFilter
import pandas as pd
import numpy as np
import json

In [4]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder

In [37]:
df = pd.read_csv('./data/20220319_covid_merge_processed.csv', sep=",")
df.head()

,location,new_cases_per_million,y,population_cov,life_expectancy_cov,Population,Area_km2,Density_km2,Year_x,Meningitis,...,Cardiovascular diseases,Chronic kidney disease,Chronic respiratory diseases,Cirrhosis and other chronic liver diseases,Digestive diseases,Acute hepatitis,Alzheimer's disease and other dementias,Parkinson's disease,Total,air_polution
0,0,4171.944,186.538,39835428.0,64.83,37171921.0,252071.0,57.0,2019.0,0.004205,...,0.166779,0.015165,0.019052,0.010215,0.018640,0.005219,0.004775,0.001507,0.585498,52.99812
1,1,92109.327,1176.434,2872934.0,78.57,2882740.0,11100.0,100.0,2019.0,0.000451,...,0.447630,0.011413,0.028272,0.011552,0.018281,0.000035,0.031810,0.008603,0.771315,18.45981
2,2,5782.052,148.940,44616626.0,76.88,42228408.0,919590.0,18.0,2019.0,0.000691,...,0.231908,0.019421,0.017827,0.009579,0.015151,0.000613,0.012335,0.003038,0.455347,32.91557
3,3,473136.492,1900.384,77354.0,83.73,77006.0,181.0,165.0,2019.0,0.000000,...,0.219463,0.020778,0.050645,0.015583,0.035062,0.001299,0.046750,0.009090,0.762278,9.51265
4,4,2898.709,55.799,33933611.0,61.15,30809787.0,481351.0,25.0,2019.0,0.008179,...,0.083493,0.007997,0.012769,0.018390,0.029117,0.000798,0.003710,0.000867,0.562613,28.02152


In [43]:
columns = df.columns.str.lower()

In [42]:
if 'y' in df.columns:
    print('y')

y


In [44]:
if 'y' not in columns:
    print('problem')

In [108]:
def test_knockoff(param):
    #df = pd.read_csv('./data/sample.csv')

    url = 'http://127.0.0.1:8000/api/v1/knockoff/filter'
    fp = './data/20220319_covid_merge_processed.csv'

    files = {'file': open(fp,'rb')} # for sending a single file
    
    # param = {'sep':',', 'fdr': 0.2, 'itr': 100}
    # token ={"name": "foo", "point": 0.13, "is_accepted": False}
    resp = requests.post(url=url, data=param, files=files)

    return resp
 


In [112]:
%%time
runs = 50
fdrs = [0.1, 0.2, 0.3, 0.4]
itrs = [10, 20, 50, 100, 200, 500, 1000]

for run in range(1, runs):
    for fdr in fdrs:
        for itr in itrs:
            param = {'sep':',', 'fdr': fdr, 'itr': itr}
            json_resp = test_knockoff()            
            with open("20220319_covid_merge_processed_results" + str(fdr) + "_" + str(itr) + "_" + str(run) +".json", "w") as outfile:
                json.dump(json_resp.json(), outfile)

CPU times: user 5.42 s, sys: 1.37 s, total: 6.79 s
Wall time: 18h 10min 48s


In [110]:
json_resp

<Response [200]>

### Post analysis

In [99]:
main_path = './results/fdr_10'

with open("20220319_covid_merge_processed_results_1000_3.json", "w") as outfile:
    json.dump(json_resp.json(), outfile)

In [22]:
with open('sample.json', 'r') as openfile:
 
    # Reading from json file
    json_object = json.load(openfile)


print(type(json_object))

<class 'str'>
